In [4]:
import time
from playwright.async_api import async_playwright
from pathlib import Path
from itertools import permutations
from datetime import datetime, timedelta
import random
import asyncio
import pickle
import json
import requests
import traceback
global counter
counter = 0

In [5]:
BOT_TOKEN = "7947071680:AAG7bE_enUmnrU84CXXlif_2CLjs3mC2e1k"
CHAT_ID = "8069197591"

In [6]:
async def save_page(page, filename):
    content = await page.content()
    save_path = Path("HTML") / filename
    save_path.write_text(content, encoding="utf-8")
    print(f"[✔] Saved: {filename}")

async def select_date(page, target_date, date_type):
    if date_type == "departure":
        await page.click("#main-search-form div.AmJm-start div div")
    await asyncio.sleep(random.uniform(2, 3))
    formatted_date = target_date.strftime("%B %e, %Y")
    formatted_date=formatted_date.replace("  "," ")

    try:
        await page.locator(f"div[role='button'][aria-label^='{formatted_date}']").click()
    except:
        await page.click('body > div.FqLu.FqLu-mod-layer-dropdown > div > div.Gagx-content > div.sGVi.sGVi-dropdown-content > div.OV9e > div.OV9e-cal-wrapper > div > div.OV9e-month-nav > div:nth-child(1)')
        try:
            await page.locator(f"div[role='button'][aria-label^='{formatted_date}']").click()
        except:
            await page.click('<svg viewBox="0 0 200 200" width="1.25em" height="1.25em" xmlns="http://www.w3.org/2000/svg" role="presentation"><path d="M80.762 164.801l-11.523-9.603L115.237 100L69.238 44.801l11.523-9.603c53.425 64.11 55.449 63.064 50 69.603l-49.999 60z"></path></svg>')
            try:
                await page.locator(f"div[role='button'][aria-label^='{formatted_date}']").click()
            except:
                return False

async def human_like_interaction(page):
    for _ in range(5):
        await page.mouse.wheel(0, random.randint(100, 300))
        await asyncio.sleep(random.uniform(1, 3))  # Randomized delay

    # Move the mouse randomly
    for _ in range(10):
        x, y = random.randint(100, 900), random.randint(100, 600)
        await page.mouse.move(x, y, steps=random.randint(5, 15))  # Smooth movement
        await asyncio.sleep(random.uniform(0.5, 1.5))

def generate_filename(from_city, to_city, TTT, LOS):
    TTT_number = (TTT.date() - datetime.today().date()).days
    departure_date = TTT.strftime("%d.%m.%y")
    return_date = (TTT + timedelta(days=LOS)).strftime("%d.%m.%y")
    return f"{from_city}_to_{to_city}_TTT_{TTT_number}_LOS_{LOS}_{departure_date}-{return_date}.html"

async def search_flight(page, from_city, to_city, TTT, LOS):
    global counter
    start_time = time.time()
    await page.goto("https://www.kayak.com")
    await asyncio.sleep(random.uniform(1, 3))
    return_date = TTT + timedelta(days=LOS)
    
    print(f"[🔍] Searching: {from_city} → {to_city} | Depart: {TTT} | Return: {return_date}")
    
    await asyncio.sleep(random.uniform(2, 3))
    buttons = [
    "#main-search-form div.J_T2-field-group div div:nth-child(1) div div div div div.c_neb-item-close div svg",
    "#main-search-form div.J_T2-field-group div div:nth-child(3) div div div div div.c_neb-item-close div svg"] 
    random.shuffle(buttons)
    for selector in buttons:
        button = await page.query_selector(selector)
        if button:
            await button.click()
            await asyncio.sleep(random.uniform(1, 2))
    counter = counter + 1
    """if  counter==15:
        await human_like_interaction(page)
    
    if  counter==50:
        pause_time = random.randint(180, 300)  
        print(f"[⏸] Taking a break for {pause_time} seconds...")
        await asyncio.sleep(pause_time)
        counter = 0"""
    
    fill_actions = [
    lambda: page.fill("#main-search-form div.J_T2-field-group div div:nth-child(1) div div input", from_city),
    lambda: page.fill("#main-search-form div.J_T2-field-group div div:nth-child(3) div div input", to_city)]

    # Shuffle the order
    random.shuffle(fill_actions)

    # Execute in random order
    for action in fill_actions:
        await action()
        await asyncio.sleep(random.uniform(1, 2))
        await page.keyboard.press("Enter")
    
    await asyncio.sleep(random.uniform(1, 2))
    await select_date(page, TTT, "departure")
    await asyncio.sleep(random.uniform(1, 2))
    await select_date(page, return_date, "return")
    await asyncio.sleep(random.uniform(1, 2))
    new_page_info = None
    try:
        async with page.context.expect_page(timeout=2000) as new_page_info:
            await page.click("button[type='submit']")
    except:
        new_page_info = None  # No new tab opened

    # Check if a new tab was detected
    if new_page_info:
        new_tab = await new_page_info.value
        temp = page
        page = new_tab  # Switch to the new tab
        print("[🌍] Switched to the new tab!")
    else:
        print("[🔄] No new tab opened. Continuing on the same page.")

    await asyncio.sleep(random.uniform(6, 7))
    
    for j in range(1, 9):
        try:
            await page.locator('#flight-results-list-wrapper > div.ULvh > div').wait_for(state='visible', timeout=50000)
        except:
            await page.screenshot(path="last_page.png")
            print(f"[❌] Timeout error. Check last_page.png for more information.")
        await page.click('#flight-results-list-wrapper > div.ULvh > div')
        await asyncio.sleep(random.uniform(5, 7))
    
    filename = generate_filename(from_city, to_city, TTT, LOS)
    await save_page(page, filename)
    print(f"[📦] Finished: {filename}")
    
async def estimate_remaining_time(start_time, current_iteration, total_iterations):
    elapsed_time = time.time() - start_time  # Time spent so far
    avg_time_per_iteration = elapsed_time / max(current_iteration, 1)  # Avoid division by zero
    remaining_iterations = total_iterations - current_iteration
    estimated_time_left = remaining_iterations * avg_time_per_iteration
    return estimated_time_left

def send_telegram_notification(error_message):
    message = f"🚨 Script Crashed!\n\n{error_message}"
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}

    try:
        requests.post(url, json=payload)
        print("📩 Telegram notification sent!")
    except Exception as e:
        print("Failed to send Telegram message:", e)

In [7]:

while True:
    try:
        async with async_playwright() as p:
            USER_AGENTS = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5359.124 Safari/537.36",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",]
            browser = await p.chromium.launch(headless=False)
            context = await browser.new_context(user_agent=random.choice(USER_AGENTS))
            page = await context.new_page()
            destinations = ["Paris", "Rome",]
            today = datetime.today()
            
            """search_queries = [
            (from_city, to_city, (today + timedelta(days=days_ahead)).strftime("%Y-%m-%d"), LOS)
            for from_city, to_city in permutations(destinations, 2)
            for days_ahead in range(1,31)
            for LOS in range(1, 6)]"""
            with open('NeedToCompleteFlight.pkl', 'rb') as f:
                search_queries = pickle.load(f)
            total_iterations = len(search_queries)
            random.shuffle(search_queries)

            progress_file = "completed_searches.json"
            try:
                with open(progress_file, "r") as file:
                    completed_searches = set(tuple(entry) for entry in json.load(file))
            except (FileNotFoundError, json.JSONDecodeError):
                completed_searches = set()  # If no file, start fresh
            completed_iterations = len(completed_searches)
            # Function to save completed searches
            def save_progress(search):
                completed_searches.add(search)
                with open(progress_file, "w") as file:
                    json.dump(list(completed_searches), file, indent=4)
            
            start_time=time.time()

            for current_iteration, search in enumerate(search_queries, start=1):  # Track iterations
                if search in completed_searches:
                    continue
                from_city, to_city, TTT, LOS = search
                """if counter==11:
                    raise Exception("Manual break")"""
                    
                await search_flight(page, from_city, to_city, datetime.strptime(TTT, "%Y-%m-%d"), LOS)
                await asyncio.sleep(random.uniform(1, 3))  # Simulate search delay

                completed_iterations += 1  # Update completed searches count
                estimated_time_left =await estimate_remaining_time(start_time, completed_iterations, total_iterations)

                hours, minutes = divmod(int(estimated_time_left), 3600)
                minutes = minutes // 60  # Convert remaining seconds to minutes

                percent_complete = (completed_iterations / total_iterations) * 100

                print(f"[✅] {percent_complete:.2f}% completed | Estimated time left: {hours}h {minutes}m")
                # Save progress
                save_progress(search)
                
    except Exception as e:
        print(f"[❌] Error: {e}")
        if str(e) == "Manual break":
            print("Manual break initiated. Exiting...")
            await browser.close()
            time.sleep(random.randint(60, 120))
            continue
            
        error_message = traceback.format_exc()
        send_telegram_notification(error_message)
        await browser.close()
        time.sleep(random.randint(60, 120))
        if(len(search_queries)==completed_iterations):
            exit

    



    
    """for from_city, to_city in permutations(destinations, 2):
        for days_ahead in range(9, 13):
            TTT = today + timedelta(days=days_ahead)
            for LOS in range(1, 6):
                current_iteration += 1
                await search_flight(page, from_city, to_city, TTT, LOS, total_iterations, current_iteration)
                await asyncio.sleep(random.uniform(2, 5))"""
    
    


[🔍] Searching: Paris → Rome | Depart: 2025-04-01 00:00:00 | Return: 2025-04-03 00:00:00
[🔄] No new tab opened. Continuing on the same page.
[✔] Saved: Paris_to_Rome_TTT_22_LOS_2_01.04.25-03.04.25.html
[📦] Finished: Paris_to_Rome_TTT_22_LOS_2_01.04.25-03.04.25.html
[✅] 12.50% completed | Estimated time left: 0h 10m
[🔍] Searching: Rome → Paris | Depart: 2025-03-26 00:00:00 | Return: 2025-03-29 00:00:00
[🔄] No new tab opened. Continuing on the same page.
[✔] Saved: Rome_to_Paris_TTT_16_LOS_3_26.03.25-29.03.25.html
[📦] Finished: Rome_to_Paris_TTT_16_LOS_3_26.03.25-29.03.25.html
[✅] 25.00% completed | Estimated time left: 0h 10m
[🔍] Searching: Paris → Rome | Depart: 2025-04-04 00:00:00 | Return: 2025-04-09 00:00:00
[🔄] No new tab opened. Continuing on the same page.
[✔] Saved: Paris_to_Rome_TTT_25_LOS_5_04.04.25-09.04.25.html
[📦] Finished: Paris_to_Rome_TTT_25_LOS_5_04.04.25-09.04.25.html
[✅] 37.50% completed | Estimated time left: 0h 8m
[🔍] Searching: Paris → Rome | Depart: 2025-04-05 00:0

CancelledError: 